# Projeto 1 - Ciência dos Dados

Nome: Enzo Qental

Nome: João Pedro Artese de Natal

Nome: João Pedro Ramos Fiorin

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [475]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

In [476]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\jprfi\OneDrive\Área de Trabalho\CDADOS\Projeto 1\CDADOS-P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [477]:
filename = 'Iphone1250.xlsx'

In [478]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,"eu achava iphone caro,até pagar o preço de nun...",1
1,"novamente sem carregador, essa vida usuária de...",2
2,se fôssemos trocar de celular vocês ficariam c...,2
3,"@zaynikr mas gente, ela tem uns 3 telefones co...",2
4,esse face id do iphone que se eu tiver com a b...,3


In [479]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,"voltei c iphone, que sdd que eu estava meu deus",1
1,o novo ios do iphone ta muito bom,3
2,"so pq tem um iphone, é triste viu kkkkkjjjjj h...",0
3,carregador de iphone seria perfeito se fosse m...,3
4,ela vive dando reply nos story usando iphone k...,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Celular da Apple. Consideramos como uma postagem relevante quando o conteúdo se trata de alguma crítica ou análise sobre o produto, caso o contrário, consideramos como não relevante.

___
### Montando as funções de limpeza dos twits.

Criando as funções que irão limpar os nossos dados das duas planilhas(Treinamento e Teste).

In [480]:
def cleanup(text):
    punctuation = '[!-.:?;()\'",_%$\\|/,<>[\]]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text).lower()
    return [p for p in text_subbed.split() if '@' != p [0]]

In [481]:
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train

,Treinamento,Relevância
0,"[eu, achava, iphone, caro, até, pagar, o, preç...",1
1,"[novamente, sem, carregador, essa, vida, usuár...",2
2,"[se, fôssemos, trocar, de, celular, vocês, fic...",2
3,"[mas, gente, ela, tem, uns, 3, telefones, como...",2
4,"[esse, face, id, do, iphone, que, se, eu, tive...",3
...,...,...
995,"[iphone, xr, 89, de, bateria, amigo, está, ven...",0
996,"[meu, namorado, me, deu, um, iphone, com, a, p...",1
997,"[a, saudade, que, to, do, meu, iphone, é, surr...",1
998,"[queria, criar, meu, vsco, logo, mas, iphone, ...",2


In [482]:
test['Teste'] = test['Teste'].apply(cleanup)
test

,Teste,Relevância
0,"[voltei, c, iphone, que, sdd, que, eu, estava,...",1
1,"[o, novo, ios, do, iphone, ta, muito, bom]",3
2,"[so, pq, tem, um, iphone, é, triste, viu, kkkk...",0
3,"[carregador, de, iphone, seria, perfeito, se, ...",3
4,"[ela, vive, dando, reply, nos, story, usando, ...",2
...,...,...
245,"[como, instalar, o, whatsapp, gb, watusi, no, ...",0
246,"[minha, vida, mudou, depois, que, o, vinicius,...",1
247,"[a, bateria, do, meu, celular, tá, parecendo, ...",3
248,"[narcisa, me, dá, um, iphone, 🥺]",0


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [483]:
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = [0,1,2,3]

In [484]:
# Separando os tweets por relevância.
muito_relevante = train['Relevância'] == 3
relevante = train['Relevância'] == 2
irrelevante = train['Relevância'] == 1
muito_irrelevante = train['Relevância'] == 0

In [485]:
# Separando os tweets por relevância em variáveis.
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

In [486]:
# Colocando todas as palavras, por categoria, em uma lista.
total_palavras_mr = sum(df_muito_relevante['Treinamento'], [])
total_palavras_r = sum(df_relevante['Treinamento'], [])
total_palavras_i = sum(df_irrelevante['Treinamento'], [])
total_palavras_mi = sum(df_muito_irrelevante['Treinamento'], [])

In [487]:
# Transformando as palavras da lista em um Pandas Series. 
series_mr = pd.Series(total_palavras_mr)
series_r = pd.Series(total_palavras_r)
series_i = pd.Series(total_palavras_i)
series_mi = pd.Series(total_palavras_mi)

In [488]:
# Calculando o total de palavras, sem repetições, de todas as categorias.
tot_palavras_sem_rep = len(series_mr.value_counts()) + len(series_r.value_counts()) + len(series_i.value_counts()) + len(series_mi.value_counts())

In [489]:
# Total de palavras por categoria. 
total_mr = len(total_palavras_mr)
total_r = len(total_palavras_r)
total_i = len(total_palavras_i)
total_mi = len(total_palavras_mi)

In [490]:
# Probabilidade de cada categoria: 
prob_mr = len(df_muito_relevante)/len(train["Treinamento"])
prob_r = len(df_relevante)/len(train["Treinamento"])
prob_i = len(df_irrelevante)/len(train["Treinamento"])
prob_mi = len(df_muito_irrelevante)/len(train["Treinamento"])

In [491]:
import numpy

def classifica(frase):
    alfa = 1
    lista_mr = []
    lista_r = []
    lista_i = []
    lista_mi = []
    
    for palavra in frase:
        qtd_mr = series_mr.to_list().count(palavra)
        qtd_r = series_r.to_list().count(palavra)
        qtd_i = series_i.to_list().count(palavra)
        qtd_mi = series_mi.to_list().count(palavra)

        prob_palavra_mr = lista_mr.append((qtd_mr + alfa) / (total_mr + tot_palavras_sem_rep))
        prob_palavra_r = lista_r.append((qtd_r + alfa) / (total_r + tot_palavras_sem_rep))
        prob_palavra_i = lista_i.append((qtd_i + alfa) / (total_i + tot_palavras_sem_rep))
        prob_palavra_mi = lista_mi.append((qtd_mi + alfa) / (total_mi + tot_palavras_sem_rep))
            
    prob_total_mr = numpy.prod(lista_mr) * prob_mr
    prob_total_r = numpy.prod(lista_r) * prob_r
    prob_total_i = numpy.prod(lista_i) * prob_i
    prob_total_mi = numpy.prod(lista_mi) * prob_mi
    
    lista_prob_total = [prob_total_mr,prob_total_r,prob_total_i,prob_total_mi]
    
    if max(lista_prob_total) == prob_total_mr:
        return 3
    elif max(lista_prob_total) == prob_total_r:
        return 2
    elif max(lista_prob_total) == prob_total_i:
        return 1
    elif max(lista_prob_total) == prob_total_mi:
        return 0

In [492]:
train['Classificacao'] = train['Treinamento'].apply(classifica)

In [493]:
tabela = pd.crosstab(train['Relevância'], train['Classificacao'], normalize='all')
tabela

Classificacao,0,1,2,3
Relevância,,,,
0,0.427,0.003,0.000,0.000
1,0.041,0.242,0.003,0.000
2,0.044,0.014,0.150,0.000
3,0.028,0.018,0.018,0.012


In [494]:
f'{(tabela.iloc[0,0] + tabela.iloc[1,1] + tabela.iloc[2,2] + tabela.iloc[3,3])*100:.2f}%'

'83.10%'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [495]:
test['Relevância'] = test['Relevância'].astype('category')
test['Relevância'].cat.categories = [0,1,2,3]

In [496]:
test['Classificacao'] = test['Teste'].apply(classifica)

In [497]:
tabela_test = pd.crosstab(test['Relevância'], test['Classificacao'], normalize='all')
tabela_test

Classificacao,0,1,2
Relevância,,,
0,0.264,0.076,0.012
1,0.164,0.076,0.012
2,0.132,0.048,0.036
3,0.108,0.040,0.032


In [498]:
f'Nosso classificador acertou:{(tabela_test.iloc[0,0] + tabela_test.iloc[1,1] + tabela_test.iloc[2,2] + 0)*100:.2f}%'

'Nosso classificador acertou:37.60%'

___
### Concluindo

In [505]:
print(f'Porcentagem de mensagens muito relevantes e que foram classificadas como muito relevantes:{0*100:.2f}%')
print('')
print(f'Porcentagem de mensagens relevantes e que foram classificadas como relevantes:{(tabela_test.iloc[2,2])*100:.2f}%')

Porcentagem de mensagens muito relevantes e que foram classificadas como muito relevantes:0.00%

Porcentagem de mensagens relevantes e que foram classificadas como relevantes:3.60%


In [506]:
print(f'Porcentagem de mensagens muito irrelevantes e que foram classificadas como muito relevantes:{0*100:.2f}%')
print('')
print(f'Porcentagem de mensagens muito irrelevantes e que foram classificadas como relevantes:{0*100:.2f}%')
print('')
print(f'Porcentagem de mensagens relevantes e que não foram classificadas como relevantes:{(1-tabela_test.iloc[2,2])*100:.2f}%')

Porcentagem de mensagens muito irrelevantes e que foram classificadas como muito relevantes:0.00%

Porcentagem de mensagens relevantes e que não foram classificadas como relevantes:96.40%


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**